In [67]:
# Note that an API key is required to download data from SimFin servers
# A key can be obtained for free by registering on www.simfin.com
# Import SimFin library as sf which is the alias name for SimFin 
import simfin as sf
import pandas as pd

In [68]:
# Set your API key for downloading data
# You will need to create a free acount and place your own API key here 
# https://simfin.com/data/api
sf.set_api_key('JKxFKoyHNRt3JGVjD6Tp1FGy2LMejBDd')
# Setup a place (local directory) to store the data 
# Set the local directory where the data will be saved
sf.set_data_dir('C:/Users/adars/OneDrive/डेस्कटॉप/Code/CFA ML,AI')
cashflow_df=sf.load_cashflow(variant = 'quarterly', market = 'us').reset_index()
balance_df=sf.load_balance(variant = 'quarterly', market = 'us').reset_index()
income_df=sf.load_income(variant = 'quarterly', market = 'us').reset_index()

Dataset "us-cashflow-quarterly" on disk (0 days old).
- Loading from disk ... 

c:\Users\adars\AppData\Local\Programs\Python\Python311\Lib\site-packages\simfin\load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
Dataset "us-balance-quarterly" on disk (0 days old).
- Loading from disk ... 

c:\Users\adars\AppData\Local\Programs\Python\Python311\Lib\site-packages\simfin\load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
Dataset "us-income-quarterly" on disk (0 days old).
- Loading from disk ... 

c:\Users\adars\AppData\Local\Programs\Python\Python311\Lib\site-packages\simfin\load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!


In [69]:
# Download the list of companies & industries

industries_df=sf.load_industries(index='IndustryId')
companies_df=sf.load_companies(index='Ticker').reset_index()

Dataset "industries" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-companies" on disk (0 days old).
- Loading from disk ... Done!


c:\Users\adars\AppData\Local\Programs\Python\Python311\Lib\site-packages\simfin\load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,
c:\Users\adars\AppData\Local\Programs\Python\Python311\Lib\site-packages\simfin\load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


In [70]:
# Let's merge industries and companies information using the left join
# Note that left join returns all rows from the left DataFrame, and any rows with matching keys from the right DataFrame. 
# When there is no matching from right DataFrame, NaN will be returned

companies_industries_df = pd.merge(companies_df, industries_df, on = 'IndustryId', how = 'left')
companies_industries_df

,Ticker,SimFinId,Company Name,IndustryId,Sector,Industry
0,A,45846,AGILENT TECHNOLOGIES INC,106001.0,Healthcare,Medical Diagnostics & Research
1,A18,1253413,Trip.com Group Ltd,NaN,NaN,NaN
2,A21,1333027,Li Auto Inc.,NaN,NaN,NaN
3,AA,367153,Alcoa Corp,110004.0,Basic Materials,Metals & Mining
4,AAC,7962652,Ares Acquisition Corporation,104002.0,Financial Services,Banks
...,...,...,...,...,...,...
5066,ZVIA,11820271,Zevia PBC,102005.0,Consumer Defensive,Beverages - Non-Alcoholic
5067,ZVO,901866,Zovio Inc,102006.0,Consumer Defensive,Education
5068,ZY,1243193,Zymergen Inc.,106002.0,Healthcare,Biotechnology
5069,ZYNE,901704,"Zynerba Pharmaceuticals, Inc.",106002.0,Healthcare,Biotechnology


In [71]:
# Now let's merge all Pandas DataFrames together
# Let's drop duplicate columns across the 3 financial statements 
cashflow_df = cashflow_df.drop(columns = ['Report Date', 
                                          'Currency', 
                                          'Publish Date', 
                                          'Restated Date', 
                                          'Shares (Basic)', 
                                          'Shares (Diluted)', 
                                          'Depreciation & Amortization'])
# Let's drop duplicate columns in the balance sheet statements
balance_df = balance_df.drop(columns = ['Report Date', 
                                        'Currency', 
                                        'Publish Date', 
                                        'Restated Date', 
                                        'Shares (Basic)', 
                                        'Shares (Diluted)'])

In [72]:
# Let's merge income statement, cashflow statement, balance sheet, EPS with the financial data
merged_df = companies_industries_df.merge(income_df, on = ['Ticker','SimFinId'], how = 'left')\
                                   .merge(cashflow_df, on = ['Ticker','SimFinId', 'Fiscal Year','Fiscal Period'], how = 'left')\
                                   .merge(balance_df, on = ['Ticker','SimFinId','Fiscal Year','Fiscal Period'], how = 'left')
merged_df

,Ticker,SimFinId,Company Name,IndustryId,Sector,Industry,Report Date,Currency,Fiscal Year,Fiscal Period,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),Revenue,Cost of Revenue,Gross Profit,Operating Expenses,"Selling, General & Administrative",Research & Development,Depreciation & Amortization,Operating Income (Loss),Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common),Net Income/Starting Line,Non-Cash Items,Change in Working Capital,Change in Accounts Receivable,Change in Inventories,Change in Accounts Payable,Change in Other,Net Cash from Operating Activities,Change in Fixed Assets & Intangibles,Net Change in Long Term Investment,Net Cash from Acquisitions & Divestitures,Net Cash from Investing Activities,Dividends Paid,Cash from (Repayment of) Debt,Cash from (Repurchase of) Equity,Net Cash from Financing Activities,Net Change in Cash,"Cash, Cash Equivalents & Short Term Investments",Accounts & Notes Receivable,Inventories,Total Current Assets,"Property, Plant & Equipment, Net",Long Term Investments & Receivables,Other Long Term Assets,Total Noncurrent Assets,Total Assets,Payables & Accruals,Short Term Debt,Total Current Liabilities,Long Term Debt,Total Noncurrent Liabilities,Total Liabilities,Share Capital & Additional Paid-In Capital,Treasury Stock,Retained Earnings,Total Equity,Total Liabilities & Equity
0,A,45846,AGILENT TECHNOLOGIES INC,106001.0,Healthcare,Medical Diagnostics & Research,2018-01-31,USD,2018.0,Q1,2018-03-06,2019-03-05,323000000.0,323000000.0,1.211000e+09,-541000000.0,670000000.0,-441000000.0,-347000000.0,-94000000.0,NaN,229000000.0,4000000.0,-11000000.0,233000000.0,NaN,233000000.0,-553000000.0,-320000000.0,NaN,-320000000.0,-320000000.0,-320000000.0,564000000.0,-80000000.0,-5000000.0,-34000000.0,-3000000.0,-38000000.0,215000000.0,-60000000.0,NaN,-6000000.0,-67000000.0,-48000000.0,135000000.0,-22000000.0,37000000.0,210000000.0,2.887000e+09,751000000.0,608000000.0,4.397000e+09,792000000.0,140000000.0,3.369000e+09,4.301000e+09,8.698000e+09,292000000.0,345000000.0,1.361000e+09,1.800000e+09,2.811000e+09,4.172000e+09,5.323000e+09,-3000000.0,-529000000.0,4.526000e+09,8.698000e+09
1,A,45846,AGILENT TECHNOLOGIES INC,106001.0,Healthcare,Medical Diagnostics & Research,2018-04-30,USD,2018.0,Q2,2018-05-31,2019-05-30,322000000.0,326000000.0,1.206000e+09,-563000000.0,643000000.0,-433000000.0,-341000000.0,-92000000.0,NaN,210000000.0,17000000.0,-9000000.0,227000000.0,NaN,227000000.0,-22000000.0,205000000.0,NaN,205000000.0,205000000.0,205000000.0,18000000.0,30000000.0,-16000000.0,0.0,-11000000.0,57000000.0,303000000.0,-48000000.0,NaN,-1000000.0,-51000000.0,-48000000.0,-30000000.0,-35000000.0,-114000000.0,122000000.0,3.011000e+09,754000000.0,594000000.0,4.525000e+09,798000000.0,139000000.0,3.322000e+09,4.259000e+09,8.784000e+09,271000000.0,315000000.0,1.365000e+09,1.800000e+09,2.802000e+09,4.167000e+09,5.335000e+09,NaN,-412000000.0,4.617000e+09,8.784000e+09
2,A,45846,AGILENT TECHNOLOGIES INC,106001.0,Healthcare,Medical Diagnostics & Research,2018-07-31,USD,2018.0,Q3,2018-08-30,2018-08-30,320000000.0,324000000.0,1.203000e+09,-542000000.0,661000000.0,-436000000.0,-339000000.0,-97000000.0,NaN,225000000.0,17000000.0,-9000000.0,242000000.0,NaN,242000000.0,-6000000.0,236000000.0,NaN,236000000.0,236000000.0,236000000.0,-5000000.0,-87000000.0,12000000.0,-32000000.0,5000000.0,-72000000.0,197000000.0,-33000000.0,NaN,-430000000.0,-472000000.0,-48000000.0,-315000000.0,-226000000.0,-589000000.0,-879000000.0,2.131000e+09,733000000.0,623000000.0,3.667000e+09,801000000.0,70000000.0,3.811000e+09,4.682000e+09,8.349000e+09,273000000.0,0.0,1.014000e+09,1.799000e+09,2.767000e+09,3.781000e+09,5.315000e+09,NaN,-416000000.0,4.568000e+09,8.349000e+09
3,A,45846,AGILENT TECHNOLOGIES INC,106001.0,Healthcare,Medical Diagnostics & Re

In [73]:
merged_df.to_csv('us_companies_quarterly_data_combined.csv', index=False)